In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import os

import datetime

import pandas as pd

from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [7]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been 

In [31]:
## def changeTimeToLocal(utcTime):
#     timeChange = datetime.timedelta(hours=7)
#     localTime = utcTime + timeChange
#     return localTime

In [32]:
# driver.quit() #close the browser

In [8]:
provinceDict = {"yala":[74,"6.506;101.341;9"],"satun":[71,"6.680;100.403;9"], "songkhla":[72,"7.125;100.658;9"],"trang":[63,"7.493;99.926;9"], \
"nakhon-si-thammarat":[65-1,"8.227;99.789;9"],"wichit":[70-1,"8.038;98.722;9"],"surat-thani":[74-1,"9.115;99.007;9"],"ranong":[71-1,"9.863;98.867;9"], \
"kanchanaburi":[57-1,"13.923;100.349;9"] , "samut-sakhon":[44-1,"13.931;100.267;9"], "bangkok":[76-1,"13.931;100.267;9"] , "samut-prakan":[42-1,"13.931;100.267;9"] , \
"mueang-nonthaburi":[35-1,"13.931;100.267;9"] , \
"pathum-thani":[36-1,"13.931;100.267;9"] , "phra-nakhon-si-ayutthaya" : [37-1,"13.931;100.267;9"]  , "saraburi":[48-1,"14.475;100.437;9"] , \
"lop-buri":[41-1,"14.475;100.437;9"] , "sing-buri":[45-1,"14.475;100.437;9"] , "prachin-buri":[54-1,"14.475;100.437;9"] , "nakhon-nayok":[32-1,"14.475;100.437;9"] , \
"chiang-rai":[1-1, "19.22;99.99;8"], "mae-hong-son":[6-1, "19.02;97.87;8"], "phayao":[4-1, "19.10;100.27;8"], "nan":[3-1, "18.79;100.80;8"], \
"chiang-mai":[2-1, "18.93;99.06;8"], "lamphun":[8-1, "18.50;99.07;8"], "lampang":[7-1, "18.35;99.42;8"], "phrae":[5-1, "18.17;100.26;8"], "uttaradit":[9-1, "17.55;100.10;8"], \
"sukhothai":[46-1, "16.96;99.97;8"], "tak":[58-1, "16.83;99.18;8"], "phitsanulok":[39-1, "16.83;100.20;8"], "kamphaeng-phet":[30-1, "16.35;99.30;8"], "phichit":[38-1, "16.37;100.45;8"], \
"phetchabun":[40-1, "16.50;101.39;8"], "nakhon-sawan":[34-1, "15.73;100.26;8"], "uthai-thani":[50-1, "15.35;100.12;8"], "nong-khai":[25-1, "17.91;102.82;8"], \
"loei":[21-1, "17.47;101.90;8"], "udon-thani":[27-1, "17.50;102.79;8"], "nakhon-phanom":[13-1, "17.41;104.73;8"], \
"sakon-nakhon":[22-1, "17.17;104.19;8"], "khon-kaen":[11-1, "16.34;102.69;8"], "kalasin":[10-1, "16.50;103.73;8"], \
"roi-et":[20-1, "16.10;103.69;8"],  "chaiyaphum":[12-1, "15.87;102.05;8"], \
"ubon-ratchathani":[28-1, "14.98;105.17;8"], "buriram":[16-1, "14.84;102.88;8"], \
"nakhon-ratchasima":[14-1,"14.85;102.04;8"], "chon-buri":[77-1, "13.25;101.13;8"], "rayong":[55-1, "12.67;101.21;8"], "trat":[52, "12.25;102.55;8"]}

In [34]:
# provinceDicct_top={"chiang-rai":[1-1, "19.22;99.99;8"], "mae-hong-son":[6-1, "19.02;97.87;8"], "phayao":[4-1, "19.10;100.27;8"], "nan":[3-1, "18.79;100.80;8"], \
#     "chiang-mai":[2-1, "18.93;99.06;8"], "lamphun":[8-1, "18.50;99.07;8"], "lampang":[7-1, "18.35;99.42;8"], "phrae":[5-1, "18.17;100.26;8"], "uttaradit":[9-1, "17.55;100.10;8"], \
#     "sukhothai":[46-1, "16.96;99.97;8"], "tak":[58-1, "16.83;99.18;8"], "phitsanulok":[39-1, "16.83;100.20;8"], "kamphaeng-phet":[30-1, "16.35;99.30;8"], "phichit":[38-1, "16.37;100.45;8"], \
#     "phetchabun":[40-1, "16.50;101.39;8"], "nakhon-sawan":[34-1, "15.73;100.26;8"], "uthai-thani":[50-1, "15.35;100.12;8"], "nong-khai":[25-1, "17.91;102.82;8"], \
#     "loei":[21-1, "17.47;101.90;8"], "udon-thani":[27-1, "17.50;102.79;8"], "nakhon-phanom":[13-1, "17.41;104.73;8"], "nong-bua-lamphu":[26-1, "17.19;102.38;8"], \
#     "sakon-nakhon":[22-1, "17.17;104.19;8"], "khon-kaen":[11-1, "16.34;102.69;8"], "kalasin":[10-1, "16.50;103.73;8"], "maha-sarakham":[17-1, "16.23;103.24;8"], \
#     "roi-et":[20-1, "16.10;103.69;8"], "amnat-charoen":[29-1, "15.82;104.69;8"], "yasothon":[19-1, "15.82;104.22;8"], "chaiyaphum":[12-1, "15.87;102.05;8"], \
#     "ubon-ratchathani":[28-1, "14.98;105.17;8"], "si-sa-ket":[24-1, "15.09;104.47;8"], "surin":[23-1, "14.82;103.26;8"], "buriram":[16-1, "14.84;102.88;8"], \
#     "nakhon-ratchasima":[14-1,"14.85;102.04;8"], "chon-buri":[77-1, "13.25;101.13;8"], "rayong":[55-1, "12.67;101.21;8"], "chantha-buri":[50, "12.39;102.43;8"], "trat":[52, "12.25;102.55;8"]}

In [35]:
# cut content
# "maha-sarakham":[17-1, "16.23;103.24;8"],
# "ang-thong":[49-1,"14.475;100.437;9"]
# "phetchaburi":[60-1,"12.899;99.623;9"]
# "yasothon":[19-1, "15.82;104.22;8"],
# "surin":[23-1, "14.82;103.26;8"],
# "si-sa-ket":[24-1, "15.09;104.47;8"],
# "nong-bua-lamphu":[26-1, "17.19;102.38;8"],
# "chantha-buri":[50, "12.39;102.43;8"]
# "amnat-charoen":[29-1, "15.82;104.69;8"],

In [36]:
print(len(provinceDict))

52


In [37]:
# url = f'https://www.ventusky.com/?p=6.66;101.09;8&l=o3&t=20200720/0600'
# driver.get(url=url) 
#15.87;101.67;7
#6.66;101.09;8

In [73]:
def findStuff(date,type):
    firstTime=True
    scrapedValue = []
    for key,value in provinceDict.items():
        url = f'https://www.ventusky.com/?p={value[1]}&l={type}&t={date}/0600'
        driver.get(url=url) 
        #wait for web to load
        if(firstTime):
            time.sleep(2)
            firstTime=False
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a[href*={key}]")))
            print ("Page is ready!")
        except TimeoutException:
            print ("Loading took too much time!")
        ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
        driver.execute_script("arguments[0].click();", ozoneValue)
        try:
            # WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'a[href={key}]')))
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'a[class="qo am zh"][href="{key}"]')))
            # WebDriverWait(driver,10).until(EC.presence_of_element_located(driver.find_elements_by_css_selector(f'a[class="qo am zh"][href="{key}"]')[0]))
        except:
            print("not found href 1")
            # try:
            #     WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'a[class="qo am i zh"][href="{key}"]')))
            # except:
            #     print("not found href 2")
        time.sleep(1)
        ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')
        realOzoneValue=ozoneValue[-1]
        for e in ozoneValue:
            testValue = e.get_attribute("data-value")
            if(testValue!=None):
                realOzoneValue=e
                break
            else:
                print(testValue)
        # ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
        #print("------------------")
        # print(ozoneValue.text)
        # print(ozoneValue.get_attribute('style'))
        scrapedValue.append([value[0],realOzoneValue.get_attribute('data-value').split(" ")[0]])
        print(realOzoneValue.get_attribute('data-value').split(" ")[0])
    return scrapedValue
    

In [74]:
yearList = ["2020","2021","2022"]
monthList = ["01","02","03","04","05","06","07","08","09","10","11","12"]
dayList = ["01","08","15","22"]
def nextDay(date):
    year=date[0:4]
    month=date[4:6]
    day=date[6:8]
    if day =="22":
        day=dayList[0]
        if month != "12":
            month=monthList[monthList.index(month)+1]
        else:
            month="01"
            year=yearList[yearList.index(year)+1]
    else:
        day=dayList[dayList.index(day)+1]
    return year+month+day

In [75]:
df_struct = {
    "date":[],
    "provinceNo":[],
    "co":[]
}

dfCO = pd.DataFrame(df_struct)

In [78]:
coResult = {}
initialDate = "20200722" #1 8 15 22

date = initialDate

while date <= "20220122": #"20220122"
    scrapedValue2 = findStuff(date,"co")
    print(scrapedValue2)
    coResult[date] = scrapedValue2

    df = pd.DataFrame(
    data=scrapedValue2,
    columns=['provinceNo','co']
    )
    df['date'] = date
    first_column = df.pop('date')
    df.insert(0, 'date', first_column)
    dfCO = pd.concat([dfCO,df])

    date = nextDay(date)

Page is ready!


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:16: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:29: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')


150
Page is ready!
220
Page is ready!
140
Page is ready!
130
Page is ready!
130
Page is ready!
100
Page is ready!
90
Page is ready!
90
Page is ready!
None
90
Page is ready!
140
Page is ready!
None
230
Page is ready!
180
Page is ready!
300
Page is ready!
360
Page is ready!
None
260
Page is ready!
None
170
Page is ready!
170
Page is ready!
None
220
Page is ready!
None
120
Page is ready!
None
140
Page is ready!
120
Page is ready!
80
Page is ready!
120
Page is ready!
120
Page is ready!
110
Page is ready!
100
Page is ready!
180
Page is ready!
190
Page is ready!
160
Page is ready!
170
Page is ready!
160
Page is ready!
160
Page is ready!
190
Page is ready!
180
Page is ready!
120
Page is ready!
140
Page is ready!
130
Page is ready!
240
Page is ready!
160
Page is ready!
140
Page is ready!
160
Page is ready!
150
Page is ready!
200
Page is ready!
150
Page is ready!
140
Page is ready!
100
Page is ready!
150
Page is ready!
130
Page is ready!
140
Page is ready!
140
Page is ready!
90
Page is ready!
1

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:16: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:29: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')


160
Page is ready!
200
Page is ready!
150
Page is ready!
110
Page is ready!
100
Page is ready!
80
Page is ready!
80
Page is ready!
80
Page is ready!
None
80
Page is ready!
90
Page is ready!
None
110
Page is ready!
90
Page is ready!
130
Page is ready!
120
Page is ready!
None
80
Page is ready!
None
80
Page is ready!
90
Page is ready!
None
80
Page is ready!
None
140
Page is ready!
None
100
Page is ready!
100
Page is ready!
80
Page is ready!
110
Page is ready!
110
Page is ready!
120
Page is ready!
110
Page is ready!
100
Page is ready!
100
Page is ready!
90
Page is ready!
80
Page is ready!
80
Page is ready!
100
Page is ready!
130
Page is ready!
100
Page is ready!
80
Page is ready!
100
Page is ready!
90
Page is ready!
120
Page is ready!
80
Page is ready!
110
Page is ready!
160
Page is ready!
130
Page is ready!
140
Page is ready!
140
Page is ready!
140
Page is ready!
100
Page is ready!
120
Page is ready!
100
Page is ready!
80
Page is ready!
80
Page is ready!
80
Page is ready!
80
[[74, '160'],

...

200
Page is ready!
220
Page is ready!
200
Page is ready!
190
Page is ready!
220
Page is ready!
220
Page is ready!
280
Page is ready!
330
Page is ready!
None
410
Page is ready!
not found href 1
1950
Page is ready!
None
860
Page is ready!
330
Page is ready!
980
Page is ready!
550
Page is ready!
None
360
Page is ready!
None
250
Page is ready!
320
Page is ready!
None
340
Page is ready!
None
300
Page is ready!
None
290
Page is ready!
440
Page is ready!
150
Page is ready!
290
Page is ready!
320
Page is ready!
570
Page is ready!
430
Page is ready!
360
Page is ready!
310
Page is ready!
300
Page is ready!
740
Page is ready!
290
Page is ready!
440
Page is ready!
360
Page is ready!
400
Page is ready!
250
Page is ready!
380
Page is ready!
350
Page is ready!
440
Page is ready!
330
Page is ready!
390
Page is ready!
320
Page is ready!
320
Page is ready!
380
Page is ready!
320
Page is ready!
360
Page is ready!
370
Page is ready!
310
Page is ready!
380
Page is ready!
380
Page is ready!
290
Page is read

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:16: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:29: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')


320
Page is ready!
260
Page is ready!
260
Page is ready!
230
Page is ready!
300
Page is ready!
280
Page is ready!
300
Page is ready!
330
Page is ready!
None
670
Page is ready!
440
Page is ready!
None
770
Page is ready!
450
Page is ready!
not found href 1
1080
Page is ready!
880
Page is ready!
None
490
Page is ready!
None
410
Page is ready!
450
Page is ready!
None
560
Page is ready!
None
420
Page is ready!
None
400
Page is ready!
530
Page is ready!
290
Page is ready!
420
Page is ready!
470
Page is ready!
480
Page is ready!
420
Page is ready!
440
Page is ready!
470
Page is ready!
540
Page is ready!
720
Page is ready!
510
Page is ready!
650
Page is ready!
530
Page is ready!
660
Page is ready!
360
Page is ready!
480
Page is ready!
510
Page is ready!
450
Page is ready!
520
Page is ready!
400
Page is ready!
370
Page is ready!
420
Page is ready!
490
Page is ready!
420
Page is ready!
440
Page is ready!
560
Page is ready!
400
Page is ready!
480
Page is ready!
500
Page is ready!
410
Page is read

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:16: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:29: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')


230
Page is ready!
240
Page is ready!
230
Page is ready!
220
Page is ready!
230
Page is ready!
220
Page is ready!
250
Page is ready!
250
Page is ready!
None
560
Page is ready!
440
Page is ready!
None
1010
Page is ready!
330
Page is ready!
not found href 1
1840
Page is ready!
not found href 1
2620
Page is ready!
not found href 1
None
2660
Page is ready!
not found href 1
None
1070
Page is ready!
not found href 1
1300
Page is ready!
None
990
Page is ready!
None
350
Page is ready!
None
420
Page is ready!
450
Page is ready!
260
Page is ready!
480
Page is ready!
570
Page is ready!
550
Page is ready!
450
Page is ready!
520
Page is ready!
610
Page is ready!
630
Page is ready!
750
Page is ready!
580
Page is ready!
710
Page is ready!
740
Page is ready!
750
Page is ready!
710
Page is ready!
790
Page is ready!
890
Page is ready!
550
Page is ready!
610
Page is ready!
620
Page is ready!
440
Page is ready!
470
Page is ready!
670
Page is ready!
630
Page is ready!
620
Page is ready!
590
Page is ready!


In [79]:
dfCO.to_csv('../ScrapedData/COdata.csv')

In [22]:
df_struct = {
    "date":[],
    "provinceNo":[],
    "o3":[]
}

dfO3 = pd.DataFrame(df_struct)
o3Result={}

In [77]:
initialDate = "20211215" #1 8 15 22 20200722

date = initialDate

while date <= "20220122": #"20220122"
    scrapedValue = findStuff(date,"o3") #"20200720"
    print(scrapedValue)
    o3Result[date] = scrapedValue

    df = pd.DataFrame(
    data=scrapedValue,
    columns=['provinceNo','o3']
    )
    df['date'] = date
    first_column = df.pop('date')
    df.insert(0, 'date', first_column)
    dfO3 = pd.concat([dfO3,df])

    date = nextDay(date)


Page is ready!


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:16: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_18388\2407419497.py:29: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')


71
Page is ready!
72
Page is ready!
82
Page is ready!


KeyboardInterrupt: 

In [55]:
dfO3

,date,provinceNo,o3
0,20201015,74.0,43
1,20201015,71.0,42
2,20201015,72.0,38
3,20201015,63.0,39
4,20201015,64.0,37
...,...,...,...
47,20220122,15.0,105
48,20220122,13.0,85
49,20220122,76.0,83
50,20220122,54.0,101


In [56]:
dfO3.to_csv('../Data/O3Data_part2.4.csv')

# append O3Data_part1 กับ O3Data_part2.4 เนื่องจากเกิด error ตอน scape ทำให้เกิด 2 dataframe ขึ้น

In [57]:
O3Part1 = pd.read_csv('../Data/O3Data_part1.csv')
O3Part2 = pd.read_csv('../Data/O3Data_part2.4.csv')

In [70]:
O3Data = pd.concat([O3Part1,O3Part2])
O3Data = O3Data.drop(columns=O3Data.columns[0], axis=1)
O3Data

,date,provinceNo,o3
0,20200722,74.0,55
1,20200722,71.0,98
2,20200722,72.0,63
3,20200722,63.0,37
4,20200722,64.0,46
...,...,...,...
3219,20220122,15.0,105
3220,20220122,13.0,85
3221,20220122,76.0,83
3222,20220122,54.0,101


In [72]:
O3Data.to_csv('../ScrapedData/O3Data.csv', index=True)